# Coffee Maker code

## Arm Robot Angles Computation
See [this post on stackexchange][original idea] for the basic math behind it.
![triangle](arm_diagram.png)

[original idea]: http://math.stackexchange.com/questions/1423467/calculating-angles-neccessary-to-reach-a-position-on-a-2d-plane-for-two-robot-ar

In [ ]:
import math
#import numpy as np

class Vec2d:
    def __init__(self, x, y):
        #self.a = np.array([x, y])
        self.x = x
        self.y = y

    def __add__(self, v):
        return Vec2d(self.x + v.x, self.y + v.y)

    def __sub__(self, v):
        return Vec2d(self.x - v.x, self.y - v.y)

    def length(self):
        return math.sqrt(self.x**2+self.y**2)
        #return np.linalg.norm(np.array([self.x, self.y]))

def calculate_servo_angles(capsule_x, capsule_y):
    # origin at the arm's base
    # horizontal (x, y), vertical z
    # units are mm
    # base rotation (orientation)
    horiz_dist = Vec2d(capsule_x, capsule_y).length()
    #base_rotation_degrees = math.degrees(math.atan2(capsule_x, capsule_y)+math.atan2(-27.88, horiz_dist))
    base_rotation_degrees = math.degrees(math.atan2(-capsule_y, capsule_x)+math.acos(27/horiz_dist))
    # vertical plane:
    capsule_center_height = 15
    # the arm has 3 main parts, the upper arm, the forearm, and the hand,
    # connected by the elbow and the wrist
    # We want the hand to be at a vertical angle, and
    # have it's end on the capsule's center of mass
    # With this we calculate the wrist's position.
    # From now on we'll work on the vertical plane that goes from the arm's
    # base to the capsule with X as horizontal and Y as vertical
    hand_length = 135 # calculated from the wrist at it's target position
                      # to the capsule's center of mass
    #wrist_x = horiz_dist - math.cos(math.radians(60)) * hand_length
    wrist_x = horiz_dist + 35 # the grip is not in the same vertical as the rotation axis, hence the 35 delta
    wrist_y = capsule_center_height + hand_length
    wrist_pos = Vec2d(wrist_x, wrist_y)
    arm_base = Vec2d(0, 82.83)
    # see arm_diagram.png for reference
    # we now have a triangle formed by the arm base (shoulder), the wrist, and the elbow
    # we know the length of the upper arm and the forearm, we need the missing side and
    # then we'll be able to calculate the angles:
    upper_arm_length = 181 # a in the diagram
    forearm_length = 142 # b
    base_to_wrist = wrist_pos - arm_base
    base_to_wrist_distance = base_to_wrist.length() # c
    #print(base_to_wrist_distance)
    # law of cosines
    # c² = a² + b² - 2ab * cos(β); where β is the angle at the vertex opposite of side c
    # 2ab * cos(β) = a² + b² - c²
    # cos(β) = (a² + b² - c²) / (2ab)
    # β = acos((a² + b² - c²) / (2ab))
    # everything radians until output
    elbow_angle = math.acos((upper_arm_length**2 + forearm_length**2 - base_to_wrist_distance**2) /
                            (2 * upper_arm_length * forearm_length)) # β
    # now for the angle between the x axis and the base_to_wrist vector (lower shoulder angle)
    base_to_wrist_angle = math.atan2(base_to_wrist.y, base_to_wrist.x) # γ
    # the angle for the base (shoulder) servo will be the base_to_wrist_angle + the angle in the
    # shoulder vertex in the triangle
    upper_shoulder_angle = math.acos((upper_arm_length**2 + base_to_wrist_distance**2 - forearm_length**2) /
                            (2 * upper_arm_length * base_to_wrist_distance)) # δ
    # now for the wrist angle:
    # from our triangle we take the last angle, A = 180 - δ - β
    A = math.radians(180) - upper_shoulder_angle - elbow_angle
    # we have another triangle with vertices at shoulder, wrist and capsule,
    # with angles γ, 60 and B = 180 - 60 - γ
    B = math.radians(180 - 90) - base_to_wrist_angle
    # The wrist angle is A + B:
    wrist_angle_degrees = math.degrees(A + B)
    # therefore we should rotate the shoulder servo to this angle:
    shoulder_angle_degrees = math.degrees(upper_shoulder_angle + base_to_wrist_angle)
    # and the elbow angle to this angle:
    elbow_angle_degrees = math.degrees(elbow_angle)
    return (base_rotation_degrees, shoulder_angle_degrees, elbow_angle_degrees, wrist_angle_degrees)

# just a test
calculate_servo_angles(50, 100)

# PhysicalRobot and NXRobot

There are 2 classes with the same interface used to control the robot motors,
one used for the simulation with Siemens' NX, and one for the real servo control
on the raspberry pi.

In [ ]:
import pigpio
import time

MIN_PW = 850
MID_PW = 1450
MAX_PW = 2100
SERVO_BASE = 2 # pin 3
SERVO_SHOULDER = 3 # pin 5
SERVO_ELBOW = 4 # pin 7
SERVO_WRIST = 17 # pin 11
SERVO_WRIST2 = 27 # pin 13
SERVO_GRIP = 22 # pin 15

def servo(angle):
    return MIN_PW+angle/360*(MAX_PW-MIN_PW)

class PhysicalRobot:
    def __init__(self):
        self.pi = pigpio.pi()
        
    def set_pos(self, x, y):
        pass
    
    def set_base(self, angle):
        pi.set_servo_pulsewidth(SERVO_BASE, servo(angle))
        
    def set_shoulder(self, angle):
        pi.set_servo_pulsewidth(SERVO_SHOULDER, servo(angle))

    def set_elbow(self, angle):
        pi.set_servo_pulsewidth(SERVO_ELBOW, servo(angle))

    def set_wrist(self, angle):
        pi.set_servo_pulsewidth(SERVO_WRIST, servo(angle))

    def set_wrist2(self, angle):
        pi.set_servo_pulsewidth(SERVO_WRIST2, servo(angle))

    def set_grip(self, angle):
        pi.set_servo_pulsewidth(SERVO_GRIP, servo(angle))

In [ ]:
from opcua import Client
from opcua import ua

def nx(angle):
    return math.radians(angle)

class NXRobot:
    def __init__(self):
        print("connecting...")
        #client = Client("opc.tcp://192.168.100.182:1234/")
        #client = Client("opc.tcp://192.168.192.76:1234/")
        #client = Client("opc.tcp://192.168.192.63:1234/")
        client = Client("opc.tcp://192.168.192.56:1234/")
        client.connect()
        self.client = client
        print("connected")
        root = client.get_root_node()
        self.root = root
        self.base_var = root.get_child(["0:Objects", "0:Robot", "0:Servo5"])
        self.shoulder_var = root.get_child(["0:Objects", "0:Robot", "0:Servo0"])
        self.elbow_var = root.get_child(["0:Objects", "0:Robot", "0:Servo1"])
        self.wrist_var = root.get_child(["0:Objects", "0:Robot", "0:Servo2"])
        self.wrist2_var = root.get_child(["0:Objects", "0:Robot", "0:Servo3"])
        self.grip_var = root.get_child(["0:Objects", "0:Robot", "0:Servo4"])
        
    def set_pos(self, x, y):
        root = self.root
        xvar = root.get_child(["0:Objects", "0:Robot", "0:PosX"])
        yvar = root.get_child(["0:Objects", "0:Robot", "0:PosY"])
        xvar.set_value(x/1000)
        yvar.set_value(y/1000)
        
    def set_base(self, angle):
        self.base_var.set_value(nx(angle))
        
    def set_shoulder(self, angle):
        self.shoulder_var.set_value(nx(angle))

    def set_elbow(self, angle):
        self.elbow_var.set_value(nx(180-angle))

    def set_wrist(self, angle):
        self.wrist_var.set_value(nx(180-angle))

    def set_wrist2(self, angle):
        self.wrist2_var.set_value(nx(angle))

    def set_grip(self, angle):
        self.grip_var.set_value(nx(angle))

## Computer Vision

We use picamera to take a picture, then with opencv (cv2):

* filter by hsv,
* blur the mask,
* find the contours,
* and then the center of the contours with a minimum size

I pick the colors ranges from the picture using GIMP, opencv uses another range for hsv though.

In [ ]:
def gimp_to_cv2(color):
    h, s, v = color
    color = h * 180 / 360, s * 255 / 100, v * 255 / 100
    return color

#coffee_colors = [gimp_to_cv2(c) for c in [
#    (236, 32, 59),
#    (24, 89, 84),
#    (25, 88, 69)
#]]
coffee_colors = [gimp_to_cv2(c) for c in [
    (255, 36, 56), # blau 0
    (15, 84, 55), # taronja 1
    (13, 39, 50), # marró 2
    (305, 13, 26), # blau fosc 3
    (27, 59, 63), # groc 4
    (278, 38, 11), # blau molt fosc 5
    #(9, 28, 10), # negre 6
    (334, 16, 17),
    (55, 14, 93), # blanc ratllat 7
    (287, 62, 15), # lila fosc 8
]]

In [ ]:
import cv2
import numpy as np
import time
import picamera
import picamera.array

def cv_add(a, b):
    return tuple(a[i] + b[i] for i in range(len(a)))

def get_color(x, y):
    with picamera.PiCamera() as camera:
        camera.resolution = (2592, 1952)
        with picamera.array.PiRGBArray(camera) as stream:
            time.sleep(0.3)
            camera.capture(stream, format='bgr')
            image = stream.array
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    print(hsv[x, y])

def get_colors():
    with picamera.PiCamera() as camera:
        camera.resolution = (2592, 1952)
        with picamera.array.PiRGBArray(camera) as stream:
            time.sleep(0.3)
            camera.capture(stream, format='bgr')
            image = stream.array
            
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    i = 0
    for y, x in [(x, y) for x in range(0, 2500, 50) for y in range(0, 1900, 50)]:
        i += 1
        d = (0, 10*(i%2)+10)
        point = (x, y)
        color = ",".join(str(s) for s in list(hsv[point]))
        cv2.circle(image, point, 3, (0, 0, 0), -1)
        cv2.putText(image, color, cv_add(d, point), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 0), 1)
        
    print("looked for colors")
    cv2.imwrite("/var/www/images/pic_colors.png", image)
    
def identify_all():
    with picamera.PiCamera() as camera:
        camera.resolution = (2592, 1952)
        with picamera.array.PiRGBArray(camera) as stream:
            time.sleep(0.3)
            camera.capture(stream, format='bgr')
            image = stream.array
    print("took pic")
    image = image[0:1600, 300:2150]
    cv2.imwrite("/var/www/images/pic.png", image)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    for n, (h, s, v) in enumerate(coffee_colors):
        print(n)
        lower_color = np.array([h-10, s-50, v-50])
        upper_color = np.array([h+10, s+50, v+50])
        mask = cv2.inRange(hsv, lower_color, upper_color)

        blurred = cv2.medianBlur(mask, 15)
        #cv2.imwrite("/var/www/images/pic_blurred.png", blurred)
        #print("saved blurred pic")
        _, contours, _ = cv2.findContours(
            blurred.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_L1)

        for contour in contours:
            moments = cv2.moments(contour)
            if moments['m00'] == 0:
                continue
            point = (int(moments['m10']/moments['m00']),
                     int(moments['m01']/moments['m00']))

            area = cv2.contourArea(contour)
            color = (0, 0, 0)
            if area < 15000:
                continue
            cv2.circle(image, point, 8, color, -1)
            cv2.putText(image, str(n), point, cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)

    print("looked for centres")
    cv2.imwrite("/var/www/images/pic_all_centres.png", image)
    print("saved centres pic")

class CapsuleNotFound(Exception):
    pass

from datetime import datetime

def get_capsule_pos(coffee_type):
    t1 = datetime.now()
    with picamera.PiCamera() as camera:
        camera.resolution = (2592, 1952)
        with picamera.array.PiRGBArray(camera) as stream:
            #time.sleep(0.3)
            camera.capture(stream, format='bgr')
            image = stream.array
    print("took pic")
    image = image[0:1600, 300:2150]
    print("cropped")
    cv2.imwrite("/var/www/images/pic.png", image)
    print("saved pic")
    
    print(datetime.now() - t1)
    
    #camera = cv2.VideoCapture(0)
    #(grabbed, frame) = camera.read()
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = coffee_colors[coffee_type]
    ## define range of blue color in HSV
    lower_color = np.array([h-8, s-50, v-50])
    upper_color = np.array([h+8, s+50, v+50])
    ## TODO: actual range depending on capsule type
    #lower_color = np.array([110,50,50])
    #upper_color = np.array([130,255,255])
    ## Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_color, upper_color)
    print("colors:", lower_color, upper_color)

    print(datetime.now() - t1)
    #blurred = cv2.GaussianBlur(mask, (13, 13), 0)
    blurred = cv2.medianBlur(mask, 15)
    cv2.imwrite("/var/www/images/pic_blurred.png", blurred)
    print("saved blurred pic")

    _, contours, _ = cv2.findContours(
        blurred.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_L1)

    print(datetime.now() - t1)
    centres = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 10000:
            continue
        moments = cv2.moments(contour)
        if moments['m00'] == 0:
            continue
        point = (int(moments['m10']/moments['m00']),
                 int(moments['m01']/moments['m00']))
        centres.append(point)
        
        color = (0, 0, 0)
        cv2.circle(image, point, 8, color, -1)
        cv2.putText(image, str(area), point, cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
        cv2.putText(image, str(point), cv_add(point, (0, 50)), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
        
    print("looked for centres")
    cv2.imwrite("/var/www/images/pic_centres.png", image)
    print("saved centres pic")
    
    print(datetime.now() - t1)
    ## Robot Arm Movement
    #####################
    if not centres:
        raise CapsuleNotFound("Capsule not found")
    x, y = centres[0]
    return x, y

In [ ]:
identify_all()

In [ ]:
#import cProfile
#cProfile.run('get_capsule_pos(0)', sort="cumtime")
get_capsule_pos(2)

In [ ]:
#get_color(958, 675)
while True:
    try:
        get_capsule_pos(3)
    except CapsuleNotFound:
        pass

In [ ]:
get_colors()

In [ ]:
def make_coffee(robot, coffee_type):
    
    x, y = 100, 100
    
    # does nothing on the real robot.
    # on the simulation, put the capsule at the right place
    print("setting capsule pos")
    robot.set_pos(x, y)
    # TODO: convert camera coordinates to world coordinates (in mm)
    
    capsule_pos = Vec2d(x, y)
    nx_base = Vec2d(69.923883, 315.525137)
    capsule_pos_nx = nx_base - capsule_pos
    print("calculating capsule angles")
    angles = calculate_servo_angles(capsule_pos_nx.x, -capsule_pos_nx.y)
    
    #angles = calculate_servo_angles(x, y)
    
    base_rotation, shoulder_angle, elbow_angle, wrist_angle = angles
    print("base:", base_rotation)

    def neutral():
        "moves arm to a safe neutral position"
        print("neutral")
        robot.set_shoulder(90)
        robot.set_elbow(90)
        robot.set_wrist(90)
        robot.set_wrist2(90)
        #time.sleep(5)
    
    # first get the arm in a safe position
    neutral()
    
    # then go to the capsules
    # TODO: go over the nespresso
    robot.set_base(0)
    #time.sleep(3)
    print("moving base to capsules")
    robot.set_base(base_rotation)
    # open grip
    print("grip open")
    robot.set_grip(0)
    time.sleep(5)
    
    print("moving arm to capsule")
    robot.set_shoulder(shoulder_angle)
    robot.set_elbow(elbow_angle)
    robot.set_wrist(wrist_angle)
    time.sleep(2)
    
    print("closing grip")
    # close the grip
    robot.set_grip(32) # TODO
    time.sleep(2)

    # get arm up again
    neutral()
    time.sleep(2)
    
    print("going to nespresso")
    # TODO: go to the nespresso
    robot.set_base(0)
    robot.set_shoulder(90)
    robot.set_elbow(90)
    robot.set_wrist(90)
    #robot.set_wrist2(0)
    
    time.sleep(5)
    print("releasing capsule")
    # Open grip
    robot.set_grip(0)
    
    neutral()


## Mi5 OPC UA Module

Here we serve our skill to the Mi5 demonstrator system

In [ ]:
import logging
#from datetime import datetime
import time
import threading

try:
    from IPython import embed
except ImportError:
    print("no ipython")
    import code

    def embed():
        vars = globals()
        vars.update(locals())
        shell = code.InteractiveConsole(vars)
        shell.interact()


from opcua import ua, Server

dbgl = []

def execute_skill(node):
    print("oh oh")
    # robot = PhyiscalRobot()
    robot = NXRobot()

    skill = node.get_parent().get_display_name().Text.decode("ascii")
    n = str(skill[len("SkillInput"):])
    params = node.get_parent().get_child("4:ParameterInput")
    coffee_type = params.get_child(["4:ParameterInput0", "0:Value"]).get_value()
    out = self.server.get_root_node().get_child(
        ["4:Module2301", "4:Output", "4:SkillOutput", "4:SkillOutput"+n])
    ##
    print("coffee type:", coffee_type)
    dbgl.append(node)
    ##
    # TODO: Figure out how to use each correctly
    try:
        busy = out.get_child("0:Busy")
        done = out.get_child("0:Done")
        ready = out.get_child("0:Ready")
        #busy.set_value(True)
        done.set_value(False)
        ready.set_value(False)
        make_coffee(coffee_type, robot)
        #time.sleep(5)
        print("made coffe!")
    except Exception as e:
        error = out.get_child("0:Error")
        error_desc = out.get_child("0:ErrorDescription")
        error.set_value(True)
        error_desc.set_value(str(e))
    finally:
        #busy.set_value(False)
        done.set_value(True)
        ready.set_value(True)

class SubHandler(object):

    """
    Subscription Handler. To receive events from server for a subscription
    """

    def __init__(self, server):
        self.server = server

    def datachange_notification(self, node, val, data):
        #print("Python: New data change event", node, val)
        if node.get_display_name().Text == b"Execute" and val == True:
            print("making coffe!")
            node.set_value(False)
            t = threading.Thread(target=execute_skill, args=[node])
            t.start()
            # server.get_root_node().get_child(["2:Module2302", "2:Input"])

        else:
            print("Write! {}".format(val))
            print(node)

    def event_notification(self, event):
        print("Python: New event", event)


# method to be exposed through server


def main():    
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger("opcua.subscription_service")
    logger.setLevel(logging.WARN)

    print("creating server")
    server = Server()
    #server.disable_clock()
    #server.set_endpoint("opc.tcp://localhost:4840/freeopcua/server/")
    print("setting endpoint")
    server.set_endpoint("opc.tcp://0.0.0.0:4840/")
    server.set_server_name("FreeOpcUa Example Server")
    print("created server, adding stuff...")

    # setup our own namespace
    #uri = "http://examples.freeopcua.github.io"
    #idx = server.register_namespace(uri)
    idx = 4

    # get Objects node, this is where we should put our custom stuff
    #objects = server.get_objects_node()
    root = server.get_root_node()

    def addir(parent, name):
        return parent.add_folder(idx, name)

    to_subscribe = []
    def addvar(parent, name, value):
        nodeid = name
        pp = parent
        while True:
            n = pp.get_display_name().Text.decode("ascii")
            if n == "Root":
                nodeid = "MI5."+nodeid
                break
            nodeid = n+"."+nodeid
            pp = pp.get_parent()
        nodeid = ua.NodeId(nodeid, idx)
        v = parent.add_variable(nodeid, name, value)
        v.set_writable()
        to_subscribe.append(v)
        return v

    mod_id = 2301

    moddir = addir(root, "Module" + str(mod_id))

    addvar(moddir, "reset", False)

    indir = addir(moddir, "Input")

    # TODO: ???
    addvar(indir, "ConnectionTestInput", False)
    addvar(indir, "EmergencyStop", False)
    addvar(indir, "Mode", 1)
    addvar(indir, "PositionInput", 3000)
    addvar(indir, "Watchbit", False)

    skindir = addir(indir, "SkillInput")
    for i in range(16):
        skindir_n = addir(skindir, "SkillInput"+str(i))
        exe = addvar(skindir_n, "Execute", False)
        parin = addir(skindir_n, "ParameterInput")
        for i in range(6):
            parin_n = addir(parin, "ParameterInput"+str(i))
            sv = addvar(parin_n, "StringValue", "")
            v = addvar(parin_n, "Value", 0.)

    outdir = addir(moddir, "Output")

    addvar(outdir, "Connected", False)
    addvar(outdir, "ConnectionTestOutput", False)
    addvar(outdir, "CurrentTaskDescription", "description here")
    addvar(outdir, "Dummy", False)
    addvar(outdir, "Error", False)
    addvar(outdir, "ErrorDescription", "")
    addvar(outdir, "ErrorId", 0)
    addvar(outdir, "ID", mod_id)
    addvar(outdir, "IP", "")
    addvar(outdir, "Idle", False)
    addvar(outdir, "Name", "Coffee Module 2")
    addvar(outdir, "PositionOutput", 3750)
    addvar(outdir, "PositionSensor", False)

    skoutdir = addir(outdir, "SkillOutput")

    dummy_skill = {
        "Dummy": True,
        "ID": 6600,
        "Name" : "Dummy"
    }
    skills = [
        {
            "Dummy": False,
            "ID": 6010,
            "Name" : "Nespresso Coffee"
        }
    ]

    for i in range(16):
        skill = dummy_skill
        if i < len(skills):
            skill = skills[i]
        skoutdir_n = addir(skoutdir, "SkillOutput"+str(i))
        addvar(skoutdir_n, "Activated", True)
        addvar(skoutdir_n, "Busy", False)
        addvar(skoutdir_n, "Done", False)
        addvar(skoutdir_n, "Dummy", skill["Dummy"])
        addvar(skoutdir_n, "Error", False)
        addvar(skoutdir_n, "ID", skill["ID"])
        addvar(skoutdir_n, "Name", skill["Name"])
        addvar(skoutdir_n, "Ready", True)
        parout = addir(skoutdir_n, "ParameterOutput")
        for i in range(6):
            parout_n = addir(parout, "ParameterOutput"+str(i))
            addvar(parout_n, "Default", 0)
            addvar(parout_n, "Dummy", True)
            addvar(parout_n, "ID", 0)
            addvar(parout_n, "MaxValue", 0)
            addvar(parout_n, "MinValue", 0)
            addvar(parout_n, "Name", "Type of coffee")
            addvar(parout_n, "Required", False)
            addvar(parout_n, "Unit", "")

    statevalue = addir(outdir, "StateValue")
    for i in range(11):
        statevalue_n = addir(statevalue, "StateValue"+str(i))
        addvar(statevalue_n, "Description", "")
        addvar(statevalue_n, "Dummy", True)
        addvar(statevalue_n, "Name", "")
        addvar(statevalue_n, "Unit", "")
        addvar(statevalue_n, "Value", 0.)

    # creating an event object
    # The event object automatically will have members for all events properties
    myevgen = server.get_event_generator()
    myevgen.event.Severity = 300

    # starting!
    print("starting server")
    server.start()
    print("Available loggers are: ", logging.Logger.manager.loggerDict.keys())
    try:
        # enable following if you want to subscribe to nodes on server side
        handler = SubHandler(server)
        # 500 is the subscription interval in ms?
        sub = server.create_subscription(500, handler)
        for v in to_subscribe:
            sub.subscribe_data_change(v)
        #handle = sub.subscribe_data_change(myvar)
        # trigger event, all subscribed clients wil receive it
        #myevgen.trigger(message="This is BaseEvent")

        v = root.get_child(["4:Module2301", "4:Input", "4:SkillInput", "4:SkillInput0", "0:Execute"])
        v.set_value(True)
        #embed()
        while True:
            pass
    finally:
        server.stop()

In [ ]:
#main()

robot = NXRobot()
make_coffee(robot, 5)
print("done")
robot.client.disconnect()